# Shifting Db2 via Clone and Deploy
The Db2 Shift clone option is used to take an existing Db2 database that is currently on-premises and clone it into a directory. This cloned database can then be transported to another server and be deployed at that location.
The advantage of cloning is that the destination does not need to be connected to the source location and the deployment of the clone can be done at a more convenient time.

The Db2 Shift program requires the following information:
* Source Database details
* Clone Options

The destination details are not required to clone a database.

## Terminal Window
Before running the examples in this lab, you will need to open up a terminal window. If you have not already done so, see the <a href="./Terminal.ipynb">Opening up a Terminal Window</a> section. 

The commands that you need to copy and execute in the terminal window are surrounded with a blue box. Select all the text in the box and then use the copy keys (^C or &#x2318;C) to place the text into the clipboard.

<div class="boxed">The command you need to copy and paste into the terminal window are found in here.
</div>

When you switch to the terminal window, you will use the appropriate Paste command (^&#x21E7;v or &#x2318;v). This will load the command into the terminal window and immediately execute it.

## Identify the Source Database
The Db2 database server is we are using in this lab is found on machine `10.0.0.1` (Server 1). This server has several databases available to do a shift from. The first step is to connect to Server 1 using by using a secure shell connection. The password for db2inst1 is **db2inst1**.

<div class="boxed">ssh db2inst1@10.0.0.1
</div>

While logged in as db2inst1@server1, use the command line to determine what databases are available.

<div class="boxed">db2 list db directory
</div>

You should see five databases:
* ONLINE – Flight on time arrivals database
* REPO – Data Management Console repository
* BANKING – Banking data for machine learning
* SAMPLE – Db2 SAMPLE database
* TRADING – Simulated stock trading database

You are going to move the **ONTIME** database in this lab. 

Each chapter in this lab will use a different database. You can choose to use any of the databases in the list above, but the commands and results shown in the labs will be for the recommended database.

Let's check the number of rows in the ONTIME database.

<div class="boxed">clear
db2 "CONNECT TO ONTIME"
db2 "SELECT COUNT(*) FROM ONTIME.ONTIME"
</div>

## Clone Details
Instead of shifting the database to a Db2 POD or instance, you will be creating a clone of the database. This clone will contain all the control and database information required by the Db2 Shift program to deploy the database at another location. This process is extremely quick compared to running the shift process over the network.
The directory that you supply must be available to the server and should be transparent to the Db2 Shift program. In other words, the file system must behave like a Linux file system from a file I/O perspective. An example of a shared file system that is transparent to Db2 Shift is NFS.
For this lab you should use the value /tmp/ontime.

The information that is required for this clone operation:
* Name of the local database (`ONTIME`)
* Instance owner at target (`db2inst1`)
* Directory (`/tmp/ontime`)

### Clone Directory
 
The cloned copy of the database will be placed into the directory that is specified by this field (`/tmp/ontime`). When the database is cloned, the contents of the directory can be moved to a new location and the deploy function use to shift it to a new location.

## Start the Db2 Shift command
In the terminal window, change to the directory to `~/c2c`. This is the directory that the Db2 Shift command was installed in. Start the program without any parameters:

<div class="boxed">cd ~/c2c
./db2shift --clone
</div>

The Db2 Shift command has the ability to go directly to the menu that we want. Here the command tells Db2 Shift to show the clone menu. The Clone menu doesn’t require a lot of information other than the source database details and a location to place the copy.

![Clone](images/c2c-clone.jpg)

Update the information on this screen with the following values:
* Database Name: `ONTIME`
* Instance Owner: `db2inst1`
* Directory: `/tmp/ontime`
    
The following settings will not be used during this lab and should be set to the default values.
* Metadata: `Refresh`
* Sync: `None`
* Exclude Routines: `Off`
* Database Mode: `Online`
* Thread Count: `4`

Once you are finished entering all the details required, press the ^X (CTRL-X) key to review the command. 
Now you can execute the command wait for the clone operation to complete. Note that the ONTIME database is large, and the clone will take a few minutes to complete.
 
There are a couple of reasons that the clone might fail. These include:
* There isn't enough space on the file system to hold the contents of the clone
* There are files in the /tmp directory that were created by another user and the program cannot delete them
* The database is in a write suspend mode already

If you come across an error where the database is in write suspend mode, you may need to issue the following command to reset the database state.

<div class="boxed">db2 restart database ontime write resume
</div>

When the command completes, exit out of the Db2 Shift menu back to the command prompt.

## Deploying the Database
Now that you have a copy of the database, you can deploy it to another instance or to a POD. We are not going to move the database in this step but rather run the deploy directly from server 1.

In this Deploy step, the cloned database will be re-created in our Kubernetes POD. To view the Deploy screen, use the following syntax.

<div class="boxed">cd ~/c2c
./db2shift --deploypod
</div>

The Deploy menu is shown below.
![Deploy](images/c2c-deploypod.jpg)

## Gather Shift Details 
Before we jump into running Db2 Shift, we need to gather some details of the clone that we want to shift and the location. Here are some fields that you will need to have information on and what to look for in the Db2 Deploy dialog.
* Database Name: `ONTIME`
* Instance Owner: `db2inst1`
* Directory that contains the clone (`/tmp/ontime`)
* Target: `Kubernetes`
* Target Database: `db2oltp`
* POD Project: `db2u`
* POD Name: `c-demo-db2u-0`

The following settings will not be used during this lab and should be set to the default values.
* Metadata: `Refresh`
* HADR Option: `off`
* Exclude Reoutines: `off`
* Thread Count: `4`
* Compression: `0`

## Running the Deploy Step
Once you are finished entering all the details required, press the ^X (CTRL-X) key to review the command.
 
At this point you can run the Shift command or save it for future use. Press ^X again to run the command. The Db2 Shift program will begin the creation of the database from the clone copy. 

When the deploy succeeds, you can check the record count of the ONTIME table with the following command.

<div class="boxed">clear
kubectl exec -i c-demo-db2u-0 -n db2u -- bash << EOF
cat << EOT > /tmp/db2cmd.db2
CONNECT TO DB2OLTP;
SELECT COUNT(*) FROM ONTIME.ONTIME;
QUIT;
EOT
su - db2inst1 "db2 -tvf /tmp/db2cmd.db2"
EOF

So what went wrong? You may have noticed that the database would not let you run a select statement against the `ONTIME` table. The reason is hidden in the settings for the Db2U POD system. The next section will give you some insights into what you need to check for when shifting a database.

## Database Analysis
The deploy step appeared to work successfully in our system, but you were unable to run a select command against one of the tables. In order to figure out what is "different" between our source and target systems, we can use the `Analyze` step in the Shift and Deploy menus.

Start up the Db2 Shift command at the Deploy step using the following command.

<div class="boxed">cd ~/c2c
./db2shift --deploypod
</div>

The data from the previous run of the utility should still be on the form. Rather than executing the command, use the ^A (CTRL-A) key to begin an analysis step.

For each Shift and Clone scenario, the Db2 Shift program provides an analysis option (^A) that will check the source and target databases and provide a summary of differences that you need to be aware of. When the analysis completes you will be presented with a summary of instance settings that are different between the source and target.

![Analysis](images/c2c-analysis.jpg)

**Note:** Your analysis will be different.

The one setting that is critical in the list is the `intra_parallel` setting which is `ON` on our source system and `OFF` at the target system. Use your cursor keys to move to the line with the `intra_parallel` field and press the space bar to select it (the circle should fill in and the line will turn light blue).

Now press ^X (CTRL-X) to generate overrides. This will populate the original deploy screen with a setting in the `Overrides` field.

![Overrides](images/c2c-overrides.jpg)

Now try running the Deploy step again (^X or CTRL-X) and let it complete the process. Once the Deploy completes, try running the query again to see what happens. We will need to stop and restart the POD for the changes to take place in the settings.

<div class="boxed">clear
kubectl exec -i c-demo-db2u-0 -n db2u -- bash << EOF
su - db2inst1 "db2stop force"
EOF

Give the POD a few minutes to restart. The `db2start` process is automatically done within the POD when a Db2 failure is detected. Next run the SELECT statement below to check that our records were successfully moved over. **Note:** If the statement fails due to a connection error, re-issue the statement again. 

<div class="boxed">clear
kubectl exec -i c-demo-db2u-0 -n db2u -- bash << EOF
cat << EOT > /tmp/db2cmd.db2
CONNECT TO DB2OLTP;
SELECT COUNT(*) FROM ONTIME.ONTIME;
QUIT;
EOT
su - db2inst1 "db2 -tvf /tmp/db2cmd.db2"
EOF

## Summary
In this lab you:
* Learned how to create a clone of a database
* Move the clone to another server (optional)
* Deploy the clone into a local instance

#### Credits: IBM 2024, George Baklarz [baklarz@ca.ibm.com]